streamlit run app.py --server.port=8086 --browser.serverAddress='177.192.12.238'

### 1. Explicação do Objetivo e Motivação:
Explique o objetivo do dashboard que você está desenvolvendo e a motivação por trás da escolha dos dados e funcionalidades que serão implementados.

Base: Chegada mensal de turistas pelo Rio de Janeiro, por via Aérea, segundo continentes e países de residência permanente, entre 2006-2019
Disponível em: https://www.data.rio/documents/a6c6c3ff7d1947a99648494e0745046d/about

- Objetivo: Apresentar a dados de turismo no Rio de Janeiro.
- Motivador: Conseguir identificar o crescimento ou redução da quantidade de turistas de acordo com o país de origem.
- Funcionalidades: 
    - Filtros por período, continente e país
    - Gráfico de linha exibindo tendências
    - Gráfico de pizza com número de turistas por continente
    - Gráfico de pareto com número absoluto de turistas por país
    - Histograma por mês
    - Mapa de calor por mês e país
    - Mapa de rotas

### 2. Realizar Upload de Arquivo CSV:
Crie uma interface em Streamlit que permita ao usuário fazer o upload de um arquivo CSV contendo dados de turismo do portal Data.Rio.

In [221]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    st.write(df)

Overwriting app.py


### 3. Filtro de Dados e Seleção:
Implemente seletores (radio, checkbox, dropdowns) na interface que permitam ao usuário filtrar os dados carregados e selecionar as colunas ou linhas que deseja visualizar.

In [238]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    st.write(df) 
    


Overwriting app.py


### 4. Desenvolver Serviço de Download de Arquivos:
Implemente um serviço que permita ao usuário fazer o download dos dados filtrados em formato CSV diretamente pela interface da aplicação.

In [239]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    #Mostro a tabela filtrada
    st.write(df) 

    #Converto para CSV
    csv = df.to_csv().encode("utf-8")

    #Crio botão para baixar o arquivo csv
    st.download_button(
        label="Baixar Dados",
        data=csv,
        file_name="base.csv",
        mime="text/csv",
    )


Overwriting app.py


### 5. Utilizar Barra de Progresso e Spinners:
Adicione uma barra de progresso e um spinner para indicar o carregamento dos dados enquanto o arquivo CSV é processado e exibido na interface.

### Adicionei o spinner na etapa de upload e a barra de progresso na etapa de download

In [245]:
%%writefile app.py

import pandas as pd
import streamlit as st
import regex as re
import time

my_bar = st.progress(0, text="Carregando o Aplicativo...")
for percent_complete in range(100):
    time.sleep(0.01)
    my_bar.progress(percent_complete + 1, text="Carregando o Aplicativo...")
time.sleep(1)
my_bar.empty()

df = pd.DataFrame()
files = st.file_uploader('Upload', type='csv', accept_multiple_files=True)

list_continentes = ['África','América Central','América do Norte', 'América do Sul', 'Ásia', 'Europa', 'Oriente Médio', 'Oceania']


if files != []:
    with st.spinner('Upload em andamento...'):
        time.sleep(2)
        st.success("Upload concluído!")
    for file in files:
        data = pd.read_csv(file)
        #Primeiro obtive a identificação do ano, que está dentro de uma frase na terceira linha
        line = data.iloc[1][0]
        #Utilizo Regex para extrair o ano
        Ano = re.findall(r'\b(20[0-9][0-9])\b', line)[0]   
        #Depois trato o restante do arquivo para ficar no formato desejado    
        data = data.iloc[4:]
        data.iloc[0][0] = 'Pais'
        data.columns = data.iloc[0]
        data = data.drop(data.index[0])
        data = data.dropna()
        #Retiro a coluna e a linha de total
        data = data[data['Pais'] != 'Total']
        data = data.drop(columns='Total')
        data['Ano'] = Ano

        #Utilizei IA para criar a função que identifica o continente (Ps: Estou com muita dificuldade para criar funções lambda)
        data['Continente'] = data['Pais'].apply(
            lambda line: next((pais for pais in list_continentes if pais in line), None)
        )
        data['Continente'] = data['Continente'].fillna(method='ffill')

        #Por fim, retiro as linhas que contém o total de cada continente, deixando apenas os países
        data = data[~data['Pais'].isin(list_continentes)]

        data = data.melt(id_vars=['Continente', 'Pais', 'Ano'], var_name='Mes', value_name='Qtd')

        #Concateno em um dataframe, pois assim permito que o usuário faça a importação de diversos CSVs, um para cada ano
        df = pd.concat([df, data])

    #Crio a lista de continentes
    list_cont = df['Continente'].drop_duplicates().tolist()
    list_cont.append('Todos')

    #Pego o id da opção "Todos"
    default_cont = list_cont.index('Todos')

    #Crio o filtro de continentes
    filter_cont = st.selectbox('Filtrar Continentes', list_cont, index=default_cont)

    #Aplico a condição para só filtrar se houver seleção
    if filter_cont != 'Todos':
        df = df.loc[df['Continente'] == filter_cont]

    #Crio o filtro de colunas
    list_colunas = st.multiselect('Selecione as colunas', df.columns, df.columns)
    #Aplico o filtro
    df = df[list_colunas]

    #Mostro a tabela filtrada
    st.write(df) 

    #Converto para CSV
    csv = df.to_csv().encode("utf-8")

    #Crio botão para baixar o arquivo csv
    st.download_button(
        label="Baixar Dados",
        data=csv,
        file_name="base.csv",
        mime="text/csv",
    )

    

Overwriting app.py


### 6. Utilizar Color Picker:
Adicione um color picker à interface que permita ao usuário personalizar a cor de fundo do painel e das fontes exibidas na aplicação.

### 7. Utilizar Funcionalidade de Cache:
Utilize a funcionalidade de cache do Streamlit para armazenar os dados carregados de grandes arquivos CSV, evitando a necessidade de recarregá-los a cada nova interação.

### 8. Persistir Dados Usando Session State:
Implemente a persistência de dados na aplicação utilizando Session State para manter as preferências do usuário (como filtros e seleções) durante a navegação.

### 9. Criar Visualizações de Dados - Tabelas:
Crie uma tabela interativa que exiba os dados carregados e permita ao usuário ordenar e filtrar as colunas diretamente pela interface.

### 10. Criar Visualizações de Dados - Gráficos Simples:
Desenvolva gráficos simples (como barras, linhas, e pie charts) para visualização dos dados carregados, utilizando o Streamlit.

### 11. Criar Visualizações de Dados - Gráficos Avançados:
Adicione gráficos mais avançados (como histograma ou scatter plot) para fornecer insights mais profundos sobre os dados.

### 12. Exibir Métricas Básicas:
Implemente a exibição de métricas básicas (como contagem de registros, médias, somas) diretamente na interface para fornecer um resumo rápido dos dados carregados.